### 기본 예시: 프롬프트 + 모델 + 출력 파서


가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 

이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


### 프롬프트 템플릿의 활용

__PromptTemplate__

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다

[사용법] 
- template: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 {}는 변수를 나타냅니다.
- input_variables: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

__input_variables__

input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [4]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

In [5]:
# from_template() 메소드를 사용하여 PromptTemplate 객체 생성

# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

아하 이해함, 나중에 프롬프트 관리 및 자동화를 위해 필요한거구나

In [7]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

### Chain 생성

LCEL(LangChain Expression Language)

![image](https://github.com/khw11044/intermediate_m/assets/51473705/522cbc57-5c12-4105-a63e-c0912280133c)


여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

In [10]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model

# | 기호는 unix 파이프 연산자와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

# 이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.



### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [11]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 입력으로 받아서 패턴이나 규칙을 학습하는 과정입니다. 이 모델은 입력된 데이터를 기반으로 가중치와 편향을 조정하면서 예측을 하고, 오차를 최소화하도록 학습합니다. 이러한 과정을 반복하면서 모델은 점차적으로 정확한 예측을 할 수 있도록 학습하게 됩니다. 이러한 학습 원리를 통해 인공지능 모델은 입력된 데이터를 기반으로 문제를 해결하고 패턴을 파악할 수 있게 됩니다.', response_metadata={'token_usage': {'completion_tokens': 189, 'prompt_tokens': 33, 'total_tokens': 222}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f30e6cf6-3540-4aa1-870f-8a45a853e372-0', usage_metadata={'input_tokens': 33, 'output_tokens': 189, 'total_tokens': 222})

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아 내부적으로 학습 알고리즘을 적용하여 일정한 규칙을 찾아내는 과정입니다. 이때, 모델은 입력 데이터의 특징을 학습하여 예측이나 분류를 수행할 수 있습니다.

학습 과정은 크게 입력층, 은닉층, 출력층으로 구성된 신경망 모델을 사용하는 경우가 많습니다. 입력층에서는 데이터를 받아들이고, 은닉층에서는 데이터의 특징을 추출하고, 출력층에서는 최종 결과를 출력합니다. 

모델은 초기에는 무작위로 설정된 가중치와 편향을 가지고 있으며, 이를 학습 데이터를 통해 최적화해나가는 과정을 거칩니다. 학습 데이터를 모델에 입력하여 예측값과 실제값을 비교하고, 이 차이를 최소화하는 방향으로 가중치와 편향을 조정해가면서 모델을 학습시킵니다.

이러한 과정을 반복하면서 모델은 학습 데이터에 대해 정확한 예측을 할 수 있도록 최적화됩니다. 이후 새로운 데이터에 대해 모델을 적용하여 예측이나 분류를 수행할 수 있습니다.

### 출력파서(Output Parser)

output_parsers 모듈에 StrOutputParser() 클래스를 불러와서 사용하면 출력물을 기본 str 형태로 받아볼 수 있습니다.

출력 파서(Output Parsers)는 LLM의 출력을 받아 더 적합한 형식으로 변환하는 역할을 담당합니다. 이는 LLM을 사용하여 어떤 형태의 구조화된 데이터를 생성할 때 매우 유용 합니다.

LangChain OutputParsers의 눈에 띄는 이점 중 하나는 많은 종류의 다양한 출력 파서를 보유하고 있을 뿐만 아니라, 많은 출력 파서들이 스트리밍을 지원 한다는 것입니다.

In [17]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [18]:
chain = prompt | model


chain

PromptTemplate(input_variables=['topic'], template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000207F1018B50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000207F101A260>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [19]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 입력으로 받아서 패턴을 학습하는 과정입니다. 이 모델은 입력된 데이터를 분석하고 특정한 규칙이나 패턴을 찾아내어 출력을 생성합니다. 모델은 이러한 패턴을 찾아내기 위해 매개변수를 조정하면서 학습을 진행합니다. 이 과정을 통해 모델은 데이터를 효과적으로 처리할 수 있는 능력을 향상시키고, 새로운 데이터에 대해 정확한 예측을 할 수 있게 됩니다.', response_metadata={'token_usage': {'completion_tokens': 179, 'prompt_tokens': 33, 'total_tokens': 212}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5b000323-0dee-4082-826e-38088edf0396-0', usage_metadata={'input_tokens': 33, 'output_tokens': 179, 'total_tokens': 212})

In [20]:
chain = prompt | model | output_parser


chain

PromptTemplate(input_variables=['topic'], template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000207F1018B50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000207F101A260>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [21]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 입력으로 받아서 패턴이나 규칙을 학습하는 과정입니다. 모델은 입력된 데이터를 분석하여 일정한 패턴을 찾아내고, 이 패턴을 기반으로 새로운 데이터를 예측하거나 분류하는 능력을 향상시키는 것이 목표입니다. 이를 위해 모델은 데이터를 입력받아 가중치를 조절하고 오차를 최소화하는 방향으로 학습을 진행합니다. 데이터가 많을수록 모델이 더 정확한 예측을 할 수 있게 되는데, 이를 통해 모델은 효율적으로 학습하고 성능을 향상시킬 수 있습니다.'

## 프롬프트 템플릿 변경 및 적용하기

- 아래의 프롬프트 내용을 얼마든지 변경 하여 테스트 해볼 수 있습니다.
- model_name 역시 변경하여 테스트가 가능합니다.



In [22]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [23]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

영어 회화:
1. Hello, I'd like a table for two, please.
2. Could we see the menu, please?
3. I'll have the grilled salmon, and my friend will have the steak, medium rare.
4. Could we get a bottle of your house white wine?
5. Could we have some extra napkins, please?
6. This looks delicious, thank you!
7. Could we get the check, please?

한글 해석:
1. 안녕하세요, 두 명이서 식사할 테이블을 부탁해요.
2. 메뉴판 좀 볼 수 있을까요?
3. 저는 구운 연어를 주문할게요, 그리고 제 친구는 스테이크를 미디엄 레어로 할게요.
4. 집에서 만든 화이트 와인 한 병 주실 수 있나요?
5. 테이블에 더 많은 냅킨 좀 주실 수 있나요?
6. 맛있어 보이네요, 감사합니다!
7. 계산서 좀 부탁드릴게요.

In [24]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

영어 회화:
- Customer: Hi, I’d like to order a large pizza with pepperoni and mushrooms, please.
- Staff: Sure, would you like to add any extra cheese or toppings?
- Customer: Yes, can you add extra cheese and olives to that?
- Staff: Absolutely. Anything to drink?
- Customer: A bottle of Coke, please.
- Staff: Great. Will that be for delivery or pick-up?
- Customer: Delivery, please.
- Staff: Can I have your address and phone number?
- Customer: Sure, it's 123 Oak Street, and my phone number is 555-1234.
- Staff: Thank the you! Your total comes to $24.95. It should take about 45 minutes. We'll call you once it’s on the way.
- Customer: Perfect, thank you!

한글 해석:
- 고객: 안녕하세요, 페퍼로니와 버섯이 들어간 라지 사이즈 피자 하나 주문하고 싶어요.
- 직원: 네, 치즈 또는 토핑을 추가하시겠어요?
- 고객: 네, 치즈와 올리브 추가해 주세요.
- 직원: 알겠습니다. 음료는 필요하신가요?
- 고객: 콜라 한 병 주세요.
- 직원: 좋습니다. 배달로 할까요, 아니면 픽업 하시겠어요?
- 고객: 배달로 부탁드려요.
- 직원: 주소와 전화번호를 알려주시겠어요?
- 고객: 네, 주소는 오크 스트리트 123번지고, 전화번호는 555-1234예요.
- 직원: 감사합니다! 총 금액은 $24.95이며, 약 45분 정도 소요될 예정입니다. 배송 출발하면 전화 

# LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록, Runnable 프로토콜을 구현했습니다.

Runnable 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다. 


__표준 인터페이스에는 다음이 포함됩니다.__

- stream: 응답의 청크를 스트리밍합니다.
- invoke: 입력에 대해 체인을 호출합니다.
- batch: 입력 목록에 대해 체인을 호출합니다.

__비동기 메소드도 있습니다.__

비동기는 위 프롬프트 표준 인터페이스와 달리 단발성 프롬프트 또는 하이퍼파라메타로 이해하면 되겠다. 즉 표준 인터페이스로 설정한 모델에 영향을 주지 않는다.

- astream: 비동기적으로 응답의 청크를 스트리밍합니다.
- ainvoke: 비동기적으로 입력에 대해 체인을 호출합니다.
- abatch: 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- astream_log: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [26]:
# LCEL 문법을 사용하여 chain 을 생성합니다.

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

### stream: 실시간 출력

이 함수는 chain.stream 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(content)을 즉시 출력합니다. end="" 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, flush=True 인자는 출력 버퍼를 즉시 비우도록 합니다.

In [27]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

멀티모달은 여러 가지 다른 형태의 정보를 동시에 제공하는 시스템이다. 예를 들어 음성, 이미지, 텍스트 등을 함께 활용하여 사용자에게 정보를 전달한다. 멀티모달은 사용자 경험을 향상시키고 정보를 효과적으로 전달할 수 있는 장점이 있다.

### invoke: 호출

chain 객체의 invoke 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [28]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 인공지능 대화 모델로, 다양한 주제에 대한 대화를 자연스럽게 이어나갈 수 있습니다. 자연어 처리 기술을 이용하여 사용자와 자연스럽게 상호작용하며 채팅을 제공합니다. 사용자의 질문에 응답하고 대화를 이어가는 것을 중점으로 하며, 학습을 통해 계속 발전하고 성능을 향상시킵니다.'

### batch: 배치(단위 실행)

함수 chain.batch는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 topic 키의 값을 사용하여 일괄 처리를 수행합니다.

In [29]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT는 인공지능 기술을 활용하여 자연스러운 대화를 제공하는 챗봇 플랫폼이다. 사용자의 질문에 대답하거나 대화를 이어나가는 등 다양한 상황에서 유용하게 활용될 수 있다. ChatGPT는 학습을 통해 지식을 쌓아가며 보다 정확하고 유용한 답변을 제공할 수 있도록 발전한다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크 서비스를 제공하는 앱이다. 사용자들은 자신의 일상을 공유하고 팔로워들과 소통할 수 있다. 해시태그를 통해 관심사나 주제에 맞는 콘텐츠를 찾아볼 수 있다.']

In [32]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)


# max_concurrency 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

# config 딕셔너리는 max_concurrency 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 

# 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.


['ChatGPT는 자연어 처리 기술을 사용하여 사람과 대화를 나눌 수 있는 AI 챗봇입니다. 사용자의 질문에 대답하거나 대화를 이어나갈 수 있어 다양한 주제에 대해 대화할 수 있습니다. ChatGPT는 학습과 경험을 통해 점차적으로 더 정확하고 유창한 대화를 제공합니다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상이나 취향을 시각적으로 나타내고 소통할 수 있습니다. 해시태그를 통해 관심사나 주제별로 탐색할 수 있고, 팔로워와 좋아요 등을 통해 사회적 상호작용을 할 수 있습니다. 또한 인플루언서나 브랜드들이 광고 및 마케팅 활동을 펼치는데도 널리 활용되고 있습니다.',
 '멀티모달은 여러 가지 다른 형태의 정보를 결합하여 제공하는 시스템이다. 이는 텍스트, 음성, 이미지, 동영상 등 다양한 매체를 활용하여 사용자에게 정보를 전달하고 상호작용할 수 있는 기능을 제공한다. 멀티모달은 사용자 경험을 향상시키고 정보 전달의 효율성을 높이는 데 도움을 준다.',
 '프로그래밍은 컴퓨터에게 작업을 시키기 위해 코드를 작성하는 과정을 말합니다. 이 코드는 컴퓨터가 이해할 수 있는 언어로 작성되며, 명령어와 알고리즘으로 이루어져 있습니다. 프로그래밍을 통해 우리는 다양한 소프트웨어와 애플리케이션을 만들어내며, 기술적인 문제를 해결하고 혁신적인 아이디어를 구현할 수 있습니다.',
 '머신러닝은 컴퓨터 시스템이 데이터를 분석하고 학습하여 패턴을 발견하고 예측하는 인공지능 기술이다. 이는 통계적 모델링과 알고리즘을 사용하여 데이터에서 지식을 추출하는 과정이다. 머신러닝은 이미지 인식, 음성 인식, 자연어 처리 등 다양한 분야에서 활용되고 있다.']

### async stream: 비동기 스트림

함수 chain.astream은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

In [33]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 동영상을 업로드하고 시청할 수 있는 온라인 동영상 플랫폼이다. 
유튜브는 사용자들이 다양한 주제의 동영상을 업로드하고 시청할 수 있는 무료 서비스를 제공한다. 
또한 광고 수익을 얻을 수 있는 파트너 프로그램도 제공하여 크리에이터들이 수익을 창출할 수 있도록 돕고 있다.

### async invoke: 비동기 호출

chain 객체의 ainvoke 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 topic이라는 키와 NVDA(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.

In [34]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

In [35]:
my_process

<coroutine object RunnableSequence.ainvoke at 0x00000207FF51DF50>

In [36]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

'NVDA는 미국의 기업으로서 시각 장애인을 위한 전자 보조기기를 제조 및 판매하는 선두 기업이다. 그들의 주요 제품은 화면 판독기와 화면 확대기로, 시각 장애인들이 컴퓨터나 스마트폰을 사용할 수 있도록 도와준다. NVDA는 시각 장애인들에게 디지털 정보 접근성을 제공하여 그들의 삶의 질을 향상시키는 데 기여하고 있다.'

### async batch: 비동기 배치

함수 abatch는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 chain 객체의 abatch 메서드를 사용하여 topic 에 대한 작업을 비동기적으로 처리하고 있습니다.

In [37]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

my_abatch_process

<coroutine object RunnableSequence.abatch at 0x00000207FF51E650>

In [38]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process

['YouTube는 동영상 공유 플랫폼으로, 사용자들이 영상을 업로드하고 시청할 수 있는 서비스이다. 다양한 콘텐츠를 제공하며, 누구나 쉽게 접근하여 다양한 주제의 동영상을 즐길 수 있다. 광고 수익을 통해 크리에이터들이 수익을 창출할 수도 있다.',
 '인스타그램은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상이나 취향을 공유하고 소통할 수 있다. 또한 해시태그를 통해 관심사에 맞는 콘텐츠를 탐색하고 팔로우할 수 있다. 인기 있는 인플루언서나 브랜드도 활발하게 홍보 및 마케팅을 진행하는데 사용된다.',
 'Facebook은 전 세계적으로 가장 인기 있는 소셜 네트워킹 서비스 중 하나이며, 사용자들이 친구들과 가족들과 소통하고 콘텐츠를 공유할 수 있는 플랫폼입니다. 또한 광고 및 마케팅을 통해 비즈니스들이 고객들과 소통하고 브랜드를 홍보할 수 있는 효과적인 도구로 활용되고 있습니다. 개인 정보 보호 문제를 둘러싼 논란과 함께 사용자들의 관심을 끌고 있으며, 다양한 기능과 서비스를 지속적으로 업데이트하며 사용자들에게 새로운 경험을 제공하고 있습니다.']

### Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다. 

예를 들어, RunnableParallel을 사용할 때(자주 사전 형태로 작성됨), 각 요소를 병렬로 실행합니다.

__langchain_core.runnables__ 모듈의 RunnableParallel 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

__ChatPromptTemplate.from_template__ 메서드를 사용하여 주어진 country에 대한 **수도** 와 **면적** 을 구하는 두 개의 체인(chain1, chain2)을 만듭니다.

이 체인들은 각각 model과 파이프(|) 연산자를 통해 연결됩니다. 

마지막으로, RunnableParallel 클래스를 사용하여 이 두 체인을 capital와 area이라는 키로 결합하여 동시에 실행할 수 있는 combined 객체를 생성합니다.

In [39]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

chain1.invoke() 함수는 chain1 객체의 invoke 메서드를 호출합니다.


In [40]:
# 이때, country이라는 키에 대한민국라는 값을 가진 딕셔너리를 인자로 전달합니다.
# chain1 를 실행합니다.
# "{country} 의 수도는 어디야?"
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울입니다.'

In [41]:
# 이번에는 chain2.invoke() 를 호출합니다. country 키에 다른 국가인 미국 을 전달합니다.
# chain2 를 실행합니다.
# "{country} 의 면적은 얼마야?"
chain2.invoke({"country": "미국"})



'미국의 면적은 약 9,833,520 km² 입니다.'

In [42]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울입니다.', 'area': '대한민국의 총 면적은 약 100,363km² 입니다.'}